In [1]:
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
df = px.data.gapminder()

In [3]:
app = JupyterDash(__name__)


In [4]:
app.layout = html.Div(children = [
  dcc.Dropdown(id="year", value=2007, clearable=False,
    options=[{"label": y, "value": y} for y in df['year'].unique()]),
  dcc.Graph(id="graph", figure={}),
  dcc.Graph(id="map", figure={})
])


In [5]:
@app.callback(Output('graph', 'figure'), Input('year', 'value'))
def cb(year):
    df_year = df.query("year == @year")
    return px.scatter(df_year, x="gdpPercap", y="lifeExp", size="pop", color="continent",
          log_x=True, size_max=60, hover_name="country", height=400,
          custom_data=[df_year.index]).update_layout(dragmode='lasso')


In [6]:
@app.callback(Output('map', 'figure'), Input('year', 'value'), Input('graph', 'selectedData'))
def display_selected_data(year, selectedData):
    df_year = df.query("year == @year")
    if selectedData:
        indices = [p["customdata"][0] for p in selectedData["points"]]
        try:
            df_year = df_year.loc[indices]
        except:
            pass

    return px.scatter_geo(df_year, locations="iso_alpha",size="pop", 
                          hover_name="country", height=400, color="lifeExp")

In [7]:
if __name__ == "__main__":
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
